# Fetching Car Price Data

In [1]:
# Importing necessary libraries

from requests import get
from bs4 import BeautifulSoup
import pandas as pd
from multiprocessing import Pool
import time

In [1]:
# Creating a function to scrape a single url from autotrader, 
# and save the results to a dataframe

def scrape_one(url):
    
    start = time.time()

    # Lists to store the scraped data in
    makes = []
    models = []
    prices = []
    ratings = []
    dates = []
    types = []
    miles = []
    litres = []
    bhps = []
    transmissions = []
    fuels = []
    owners = []
    ulezs = []

    # Attempt to connect to the url
    try:
        response = get(url)
        
        html_soup = BeautifulSoup(response.text, 'html.parser')

        # Get a list of individual cars and iterate through it
        car_containers = html_soup.find_all('li', class_ = 'search-page__result')

        for container in car_containers:
            try:
                container.find("div", {"class": "js-tooltip"}).find("div", {"class": "pi-indicator js-tooltip-trigger"}).text
                rating = container.find("div", {"class": "js-tooltip"}).find("div", {"class": "pi-indicator js-tooltip-trigger"}).text.strip()
            except:
                rating = ''
            ratings.append(rating)

            if 'land' == container.h2.text.strip().title().split(' ')[0].lower() and 'rover' == container.h2.text.strip().title().split(' ')[1].lower():
                make = 'Land Rover'
                makes.append(make)

                model = container.h2.text.strip().title().split(' ')[2]
                models.append(model)

            elif 'mercedes' == container.h2.text.strip().title().split(' ')[0].lower() and 'benz' == container.h2.text.strip().title().split(' ')[1].lower():
                make = 'Mercedes-Benz'
                makes.append(make)

                model = container.h2.text.strip().title().split(' ')[2]
                models.append(model)
            else:
                make = container.h2.text.strip().title().split(' ')[0]
                makes.append(make)

                model = container.h2.text.strip().title().split(' ')[1]
                models.append(model)

            price = container.find("div", {"class": "vehicle-price"}).text[1:]
            prices.append(price)

            specs = container.find("ul", {"class": "listing-key-specs"}).find_all("li", recursive=True)

            for spec in specs:

                if spec.text.split(' ')[0].isdigit() and len(spec.text.split(' ')[0]) == 4:
                    date = spec.text.split(' ')[0]
                    dates.append(date)

                if 'mile' in str(spec):
                    mile = spec.text.split(' ')[0]
                    miles.append(mile)

                if 'l' in str(spec).lower() and str(spec.text)[:-1].replace('.', '').isnumeric() and not spec.text.split(' ')[0].isdigit():
                    litre = spec.text[:-1]
                    litres.append(litre)

                if any(x in str(spec).lower() for x in ['automatic', 'manual']):
                    transmission = spec.text
                    transmissions.append(transmission)

                if any(x in str(spec).lower() for x in ['bhp', 'ps']):
                    bhp = spec.text
                    bhps.append(bhp)

                if any(x in str(spec).lower() for x in ['petrol', 'diesel']):
                    fuel = spec.text
                    fuels.append(fuel)

                if 'owner' in str(spec).lower():
                    owner = spec.text
                    owners.append(owner.split(' ')[0])

                if 'ulez' in str(spec).lower():
                    ulez = spec.text
                    ulezs.append(ulez)


                typelist = ['hatchback', 'saloon', 'convertible', 'coupe', 'suv', 'mpv', 'estate', 'limousine',
                            'pickup']
                if any(x in str(spec).lower() for x in typelist):
                    typ = spec.text
                    types.append(typ)


            attributes = [makes, models, ratings, dates, types, miles, litres, bhps, transmissions,
                          fuels, owners, ulezs]

            # Filling in empty spaces
            for attribute in attributes:
                if len(attribute) < len(prices):
                    attribute.append('')
                
    except:
        print('oops')
    
    # Creating a dataframe from the lists
    df = ({'Make': makes,
           'Model': models,
           'Price': prices,
            'Rating': ratings,
            'Year': dates,
            'Type': types,
            'Miles': miles,
            'Litres': litres,
            'BHP': bhps,
            'Transmission': transmissions,
            'Fuel': fuels,
            'Owners': owners,
            'ULEZ': ulezs
    })
    
    end = time.time()
    print(end - start)
    
    return df

In [3]:
# Given a make and a model of car, find the urls where the data would be stored
# Note autotrader only shows the first 100 pages, which is why i is limited

def find_urls(make,model):
    
    path = 'https://www.autotrader.co.uk/car-search?sort=relevance&postcode=sw1a0aa&radius=1500&make={}&model={}&page='.format(make.replace(' ','%20'),model.replace(' ','%20').replace('+', '%2B').replace('!', '%21'))
    urls = []
    
    for i in range(1,101):
        url = path + str(i)
        urls.append(url) 
    
    return urls

I will use models with at least 250 cars in the database

In [4]:
# Creating a dictionary to store the make and model pairs

d = {
    'ford': ['b-max', 'ka', 'fusion', 'grand c-max', 'ecosport', 'focus', 'kuga', 'mondeo', 's-max',
             'mustang', 'c-max', 'fiesta', 'galaxy', 'ka+', 'ranger'],
    
    'audi': ['a4', 'e-tron', 'q5', 's3', 'q7', 'q2', 'a7', 'a6 avant', 'a5', 'a3', 'a1', 'a3 cabriolet',
             'a4 avant', 'a5 cabriolet', 'a6 saloon', 'a8', 'q3', 'q8', 's5', 'sq5', 'tt'],
    
    'bmw': ['1 series', '3 series gran turismo', '5 series', 'm3', 'x3', 'x6', 'z4', 'x5', 'x1', 'm4',
            '4 series', '2 series gran tourer', '2 series', '2 series active tourer', '3 series',
            '4 series gran coupe', '6 series', '7 series', 'i3', 'x2', 'x4'],
    
    'vauxhall': ['adam', 'crossland x', 'insignia', 'mokka', 'zafira', 'mokka x', 'astra', 'astra gtc',
                 'corsa', 'grandland x', 'meriva', 'viva', 'zafira tourer'],
    
    'volkswagen': ['amarok', 'cc', 'golf sv', 'passat', 'scirocco', 't-roc', 'touareg', 'touran', 'tiguan',
                   'sharan', 'golf', 'arteon', 'beetle', 'polo', 't-cross', 'transporter', 'up!'],
    
    'toyota': ['avensis', 'c-hr', 'verso', 'yaris', 'prius', 'corolla', 'auris', 'aygo', 'hilux', 'prius+', 'rav4'],
    
    'nissan': ['leaf', 'pulsar', 'x-trail', 'qashqai', 'navara', 'micra', 'juke', 'note'],
    
    'mercedes-benz': ['a class', 'b class', 'clk', 'gla class', 'gle class', 'slk', 'vito', 'v class',
                      'sl class', 'cls', 'cla class', 'c class', 'e class', 'glc class', 'm class', 's class',
                      'slc'],
    
    'land rover': ['discovery', 'discovery sport', 'range rover evoque', 'range rover sport', 'defender 90',
                   'defender 110', 'discovery 4', 'freelander 2', 'range rover', 'range rover velar'],
    
    'honda': ['cr-v', 'hr-v', 'jazz', 'civic'],
    
    'jaguar': ['f-pace', 'xe', 'xf', 'f-type', 'e-pace', 'i-pace', 'x-type', 'xj'],
    
    'citroen': ['c3 picasso', 'c4 cactus', 'ds3', 'grand c4 picasso', 'c4 picasso', 'c3', 'berlingo',
                'c1', 'c3 aircross', 'c4'],
    
    'hyundai': ['i20', 'ix20', 'tucson', 'ix35', 'i30', 'i10', 'i40', 'ioniq', 'kona', 'santa fe'],
    
    'volvo': ['s60', 'v40', 'v60', 'xc40', 'xc90', 'xc60', 'v90', 'v40 cross country', 's90', 'v90 cross country'],
    
    'suzuki': ['alto', 'celerio', 'sx4 s-cross', 'vitara', 'swift', 'ignis'],
    
    'porsche': ['macan', 'cayenne', 'panamera', 'cayman', 'boxster', '911'],
    
    'skoda': ['citigo', 'karoq', 'scala', 'superb', 'kodiaq', 'fabia', 'kamiq', 'octavia', 'yeti'],
    
    'peugeot': ['108', '207', '208', '308', '508', '3008', '2008', '107', '308 sw', '5008', 'partner tepee'],
    
    'mini': ['clubman', 'countryman', 'convertible', 'hatch'],
    
    'kia': ['picanto', 'soul', 'stonic', 'venga', 'sportage', 'proceed', 'niro', 'ceed', 'optima', 'rio', 'sorento'],
    
    'fiat': ['500l', 'punto', 'tipo', '500', '500x', '500c', 'grande punto', 'panda'],
    
    'mazda': ['cx-5', 'mazda2', 'mazda6', 'mx-5', 'mazda3', 'cx-3'],
    
    'mitsubishi': ['eclipse cross', 'mirage', 'shogun', 'outlander', 'l200', 'asx'],
    
    'renault': ['clio', 'kadjar', 'megane', 'zoe', 'grand scenic', 'captur', 'scenic', 'twingo']
}

In [5]:
# Create a list of all the urls needed to scrape car data (for our dictionary)

def all_urls(d):
    
    allurls = []
    
    for make in d:
        for model in d[make]:
            allurls.append(find_urls(make,model))
    
    finalurls = [item for sublist in allurls for item in sublist]
    return finalurls

In [6]:
# Scrape all urls, use pooling to speed up this process

def scrape_group(urls, num=10):
    
    p = Pool(num)
    df = p.map(scrape_one, urls)
    p.terminate()
    p.join()
    
    makes = []
    models = []
    prices = []
    ratings = []
    dates = []
    types = []
    miles = []
    litres = []
    bhps = []
    transmissions = []
    fuels = []
    owners = []
    ulezs = []

    for i in range(len(df)):
        for j in range(len(df[i])):
            try:
                makes.append(df[i]['Make'][j])
                models.append(df[i]['Model'][j])
                prices.append(df[i]['Price'][j])
                ratings.append(df[i]['Rating'][j])
                dates.append(df[i]['Year'][j])
                types.append(df[i]['Type'][j])
                miles.append(df[i]['Miles'][j])
                litres.append(df[i]['Litres'][j])
                bhps.append(df[i]['BHP'][j])
                transmissions.append(df[i]['Transmission'][j])
                fuels.append(df[i]['Fuel'][j])
                owners.append(df[i]['Owners'][j])
                ulezs.append(df[i]['ULEZ'][j])
            except:
                makes.append('')
                models.append('')
                prices.append('')
                ratings.append('')
                dates.append('')
                types.append('')
                miles.append('')
                litres.append('')
                bhps.append('')
                transmissions.append('')
                fuels.append('')
                owners.append('')
                ulezs.append('')


    # Creating a dataframe from the lists
    df = ({'Make': makes,
           'Model': models,
           'Price': prices,
            'Rating': ratings,
            'Year': dates,
            'Type': types,
            'Miles': miles,
            'Litres': litres,
            'BHP': bhps,
            'Transmission': transmissions,
            'Fuel': fuels,
            'Owners': owners,
            'ULEZ': ulezs
    })
    df = pd.DataFrame(df)
    
    return df

In [7]:
# Get all urls we need to scrape data from

urls = all_urls(d)

In [8]:
# Scraping all urls

start_whole = time.time()
df = scrape_group(urls)
end_whole = time.time()

print('The webscraper took {:.0f} seconds'.format(end_whole - start_whole))

1.9288721084594727
2.7368149757385254
5.9867188930511475
6.453972101211548
6.633526086807251
6.907946348190308
6.976764917373657
7.00313401222229
6.9962849617004395
7.163537263870239
4.7589271068573
5.651323080062866
1.6694419384002686
1.7210502624511719
1.5942070484161377
1.6244637966156006
1.8408751487731934
1.849506139755249
2.0181379318237305
2.1420540809631348
1.5102581977844238
1.7903921604156494
1.7438273429870605
1.819230079650879
1.790313959121704
1.7671172618865967
1.8075950145721436
1.7869455814361572
1.5485689640045166
1.9303810596466064
1.892707109451294
1.7866120338439941
1.7867028713226318
1.2133278846740723
1.219862937927246
1.34617018699646
3.088412046432495
4.380874156951904
4.386013984680176
4.52643084526062
4.917819023132324
4.712119817733765
4.595293045043945
4.665934085845947
5.005680322647095
4.3166258335113525
2.910343885421753
1.4920001029968262
2.193037271499634
2.435551881790161
2.0176920890808105
2.123901128768921
2.307804822921753
2.3311400413513184
2.28944

3.852963924407959
2.024765968322754
2.1769120693206787
2.3728480339050293
1.672835111618042
1.7237608432769775
1.7699120044708252
1.7134487628936768
1.580657958984375
1.6532502174377441
1.2857861518859863
1.123406171798706
1.1284451484680176
1.7510488033294678
1.8258521556854248
1.5171761512756348
1.6617372035980225
1.7285058498382568
1.7002110481262207
1.939791202545166
1.9192712306976318
1.7264249324798584
1.7016849517822266
1.0191421508789062
1.0364110469818115
1.1181042194366455
1.207369089126587
1.4695909023284912
2.036821126937866
1.6866259574890137
1.6456267833709717
1.8635289669036865
1.4352068901062012
2.0488080978393555
1.788619041442871
1.770071029663086
1.4614028930664062
1.1528489589691162
1.1834630966186523
1.3378801345825195
1.5402469635009766
1.398087978363037
1.9245529174804688
1.9331960678100586
2.427072048187256
1.922790288925171
1.3364417552947998
1.7086610794067383
1.9577980041503906
1.2118501663208008
1.3245131969451904
0.9759109020233154
0.985255241394043
1.12794

2.5254781246185303
1.5751953125
1.9391050338745117
1.609058141708374
1.5144729614257812
1.5939528942108154
1.4569108486175537
1.2361698150634766
1.18739914894104
1.4204189777374268
1.1753442287445068
1.517686128616333
1.6668429374694824
1.631422996520996
1.6571860313415527
1.7704088687896729
1.8839879035949707
1.600846767425537
1.9582469463348389
1.6346089839935303
2.0588276386260986
0.7055690288543701
1.0914740562438965
0.9303059577941895
1.4765801429748535
1.4287898540496826
1.428419828414917
1.6825480461120605
1.5550730228424072
1.6432931423187256
1.5516750812530518
1.321216106414795
1.2458250522613525
1.1647729873657227
0.9322731494903564
1.5193769931793213
1.641158103942871
1.3953168392181396
1.6881399154663086
1.4902949333190918
1.3753888607025146
1.6926929950714111
1.6270270347595215
1.6524088382720947
1.130591869354248
0.7779619693756104
1.0200631618499756
1.1429128646850586
1.3811991214752197
1.2578999996185303
1.5278680324554443
1.4193463325500488
1.5174059867858887
1.5456638

0.9628078937530518
1.1012728214263916
0.9273667335510254
1.0949077606201172
1.189054250717163
1.3233551979064941
1.4596753120422363
1.5286941528320312
1.4933419227600098
1.5673668384552002
1.4849541187286377
1.167586088180542
1.300365924835205
1.4750699996948242
1.513118028640747
1.3791439533233643
1.1139800548553467
1.2698688507080078
1.1477200984954834
1.2929108142852783
1.2233099937438965
1.1018800735473633
1.339184045791626
1.0257411003112793
1.217231273651123
1.336137056350708
1.178276777267456
1.441499948501587
1.3991219997406006
1.501206874847412
1.458786964416504
1.2855849266052246
1.219980001449585
1.4850690364837646
1.0472569465637207
0.9432370662689209
1.060755968093872
1.0627586841583252
1.3620858192443848
1.638638973236084
1.424466848373413
1.7062621116638184
1.4617249965667725
1.3452033996582031
1.4744749069213867
1.4128010272979736
1.2602341175079346
1.3785858154296875
1.4077050685882568
1.4513781070709229
1.5816528797149658
1.711803913116455
1.4654242992401123
1.6054480

1.2420060634613037
1.4880571365356445
1.3724031448364258
1.4778351783752441
1.5174379348754883
1.6090600490570068
1.6037440299987793
1.691161870956421
1.139355182647705
1.1228020191192627
1.1695890426635742
0.928947925567627
1.0671920776367188
1.011173963546753
1.7098650932312012
1.5880978107452393
1.5465972423553467
1.9682822227478027
2.0495686531066895
1.7492790222167969
2.1211841106414795
2.0168941020965576
1.4559049606323242
1.3627738952636719
0.80460524559021
0.8425860404968262
1.3460779190063477
1.5072052478790283
1.550346851348877
1.7897899150848389
1.9971659183502197
1.4277820587158203
1.9245619773864746
2.0977518558502197
2.0947320461273193
1.5394833087921143
1.9522802829742432
1.5424981117248535
1.3518493175506592
1.940938949584961
1.6344807147979736
1.4873058795928955
1.4284319877624512
1.6163828372955322
1.480233907699585
2.850372076034546
2.931100845336914
3.686378002166748
3.050968885421753
3.067870855331421
1.574902057647705
1.431250810623169
1.5900590419769287
1.7114188

...

1.5537500381469727
1.9699499607086182
1.8099429607391357
1.600085973739624
1.5679810047149658
1.4785370826721191
1.692695140838623
1.6960160732269287
1.7343721389770508
1.5652248859405518
1.6614561080932617
1.839263916015625
1.6571340560913086
1.6210010051727295
1.320802927017212
1.502363920211792
1.5115900039672852
1.449653148651123
1.3444299697875977
1.7084879875183105
1.558046817779541
1.5461711883544922
1.5420472621917725
1.415297031402588
1.8122749328613281
1.4694621562957764
1.594796895980835
1.4943711757659912
1.7208871841430664
1.5178840160369873
1.2773211002349854
1.4465148448944092
1.512322187423706
1.6021642684936523
1.389402151107788
1.3428637981414795
1.3584308624267578
1.5599915981292725
1.2983219623565674
1.2587361335754395
1.2608919143676758
1.4094939231872559
1.245225191116333
1.39263916015625
1.4226138591766357
2.139090061187744
1.6539089679718018
2.734018087387085
1.394028663635254
1.3223371505737305
1.6912319660186768
1.6946120262145996
1.5123178958892822
1.66846895

1.1467530727386475
1.0980241298675537
1.1578340530395508
1.586136817932129
1.602492094039917
1.9842960834503174
2.4151978492736816
2.4225900173187256
2.517958164215088
2.3131659030914307
1.9480109214782715
2.5018250942230225
2.439338207244873
1.8415718078613281
1.7342588901519775
1.2309560775756836
2.1742820739746094
2.327916145324707
2.0285990238189697
2.027756690979004
2.5061678886413574
2.141547918319702
2.771669864654541
2.261482000350952
2.375441074371338
2.42684268951416
1.301476240158081
1.8255431652069092
1.8288748264312744
1.4559087753295898
1.4734289646148682
1.541978120803833
1.3776557445526123
1.391829013824463
1.4220232963562012
1.4605109691619873
1.1929600238800049
1.1775529384613037
1.4097850322723389
1.5584731101989746
1.644723892211914
2.119065046310425
2.1410391330718994
2.021491050720215
2.060349941253662
1.729830265045166
2.087899923324585
1.9512147903442383
2.0254709720611572
2.2587499618530273
1.950904130935669
2.576894998550415
2.637507677078247
2.493923902511596

1.6410009860992432
1.5928418636322021
4.106442928314209
1.940119981765747
1.9949140548706055
1.86769700050354
2.023113965988159
1.2804009914398193
2.1197378635406494
2.139759063720703
1.5441741943359375
1.9152088165283203
2.0112290382385254
1.8216757774353027
1.8531038761138916
1.6667170524597168
1.5370166301727295
1.8003759384155273
1.7803969383239746
2.0035288333892822
2.015795946121216
2.8439218997955322
2.493074893951416
2.502596139907837
2.4676952362060547
2.4282188415527344
2.1554489135742188
2.7370381355285645
2.7364931106567383
2.073101043701172
2.316377878189087
1.2861599922180176
1.3545570373535156
1.5213971138000488
1.5347211360931396
1.339838981628418
1.5426051616668701
1.7873609066009521
1.990731954574585
1.9621481895446777
1.8431711196899414
1.8227691650390625
1.3296020030975342
1.8005990982055664
1.618736982345581
2.4996001720428467
1.1455249786376953
1.8342251777648926
1.2903170585632324
1.5622100830078125
1.7215461730957031
2.0744059085845947
1.8049161434173584
2.02356

2.353041887283325
2.035971164703369
2.074625015258789
1.9603660106658936
2.3130970001220703
2.316464900970459
2.042686939239502
1.9418649673461914
1.9105558395385742
1.5795958042144775
2.0076217651367188
2.1602768898010254
2.161221981048584
1.8896238803863525
1.8826420307159424
1.6473519802093506
1.5709562301635742
1.8437540531158447
2.0136771202087402
2.273571014404297
1.818354845046997
1.89900803565979
1.7669458389282227
1.7414257526397705
1.2460298538208008
1.5774662494659424
1.9318389892578125
2.3213391304016113
1.2794592380523682
1.1608598232269287
1.7458250522613525
1.416715145111084
2.0521631240844727
1.7393767833709717
1.6594839096069336
2.05309796333313
1.7719690799713135
1.7341840267181396
1.5138590335845947
1.5886621475219727
1.6472430229187012
2.455442190170288
2.3023858070373535
2.910863161087036
2.4519457817077637
2.7432608604431152
2.5890262126922607
4.6311869621276855
1.5628082752227783
3.573732852935791
1.7080273628234863
1.3585760593414307
1.4081461429595947
1.5788569

2.5429940223693848
2.2135977745056152
1.8421630859375
1.1265618801116943
0.9941720962524414
2.0686748027801514
1.9035789966583252
2.1575980186462402
2.1825509071350098
2.6810359954833984
2.7830238342285156
2.740981101989746
2.956655979156494
2.5112290382385254
4.0562028884887695
4.0043299198150635
4.263710021972656
3.3585050106048584
3.3973660469055176
3.9591479301452637
3.5227530002593994
1.3469293117523193
3.8851258754730225
2.0204389095306396
1.7898340225219727
1.7030680179595947
2.0053718090057373
1.8528199195861816
1.9866011142730713
2.4250669479370117
2.180989980697632
1.4878101348876953
1.5332400798797607
2.3404250144958496
2.3490359783172607
2.1970224380493164
2.714103937149048
2.6586291790008545
2.5576000213623047
1.896916151046753
1.8043131828308105
1.2579288482666016
1.3805389404296875
1.637524127960205
1.3379502296447754
1.1788148880004883
1.6685302257537842
1.6786880493164062
1.410902976989746
1.7474806308746338
1.7254910469055176
1.9321119785308838
1.9244248867034912
1.97

In [10]:
# Dropping null rows (where the page is empty)

df.replace('', float('NaN'), inplace=True)
df.dropna(inplace=True, how = 'all')

In [11]:
# Save the data to a csv file

df.to_csv('final_data')